In [ ]:
import numpy as np

In [1]:
class Carrot(object):
    def __init__(self, data, child_nodes=[], name="none", requires_grad=False) -> None:
        self.data = data
        self.child_nodes = child_nodes
        self.name = name
        self.grad = None
        self.requires_grad = requires_grad
        if self.requires_grad:
            self.grad = 0

    def __mul__(self, other: "Carrot") -> "Carrot":
        """
        left mul: self * other
        """
        data = self.data * other.data
        requires_grad = self.requires_grad or other.requires_grad
        child_nodes = []
        if self.requires_grad:

            def grad_wrt_self(grad):
                return grad * other.data

            child_nodes.append((self, grad_wrt_self))
        if other.requires_grad:

            def grad_wrt_other(grad):
                return grad * self.data

            child_nodes.append((other, grad_wrt_other))

        result_node = Carrot(
            data=data, child_nodes=child_nodes, name="mul", requires_grad=requires_grad
        )
        return result_node

    def __rmul__(self, other: "Carrot") -> "Carrot":
        """
        right mul: other * self
        """
        data = other.data * self.data
        requires_grad = self.requires_grad or other.requires_grad
        child_nodes = []
        if other.requires_grad:

            def grad_wrt_other(grad):
                return grad * self.data

            child_nodes.append((other, grad_wrt_other))
        if self.requires_grad:

            def grad_wrt_self(grad):
                return grad * other.data

            child_nodes.append((self, grad_wrt_self))

        result_node = Carrot(
            data=data,
            child_nodes=child_nodes,
            name="mul",
            requires_grad=requires_grad,
        )
        return result_node

    def __add__(self, other: "Carrot") -> "Carrot":
        """
        left add: self + other
        """
        data = self.data + other.data
        requires_grad = self.requires_grad or other.requires_grad
        child_nodes = []
        if self.requires_grad:

            def grad_wrt_self(grad):
                return grad * 1.0

            child_nodes.append((self, grad_wrt_self))
        if other.requires_grad:

            def grad_wrt_other(grad):
                return grad * 1.0

            child_nodes.append((other, grad_wrt_other))

        result_node = Carrot(
            data=data,
            child_nodes=child_nodes,
            name="add",
            requires_grad=requires_grad,
        )
        return result_node

    def __radd__(self, other: "Carrot") -> "Carrot":
        """
        right add: other + self
        """
        data = self.data + other.data
        requires_grad = self.requires_grad or other.requires_grad
        child_nodes = []
        if other.requires_grad:

            def grad_wrt_other(grad):
                return grad * 1.0

            child_nodes.append((other, grad_wrt_other))
        if self.requires_grad:

            def grad_wrt_self(grad):
                return grad * 1.0

            child_nodes.append((self, grad_wrt_self))

        result_node = Carrot(
            data=data,
            child_nodes=child_nodes,
            name="add",
            requires_grad=requires_grad,
        )
        return result_node

    def __pow__(self, n) -> "Carrot":
        """
        self ** n
        """
        data = self.data**n
        child_nodes = []
        if self.requires_grad:

            def grad_wrt_self(grad):
                return grad * n * self.data ** (n - 1)

            child_nodes.append((self, grad_wrt_self))

        result_node = Carrot(
            data=data,
            child_nodes=child_nodes,
            name="pow",
            requires_grad=self.requires_grad,
        )
        return result_node

    def backward(self, grad=None):
        """
        backward pass
        """
        if grad is None:
            self.grad = 1.0
            grad = 1.0
        else:
            self.grad += grad
            pass
        # recursion for bp
        for child_node, grad_wrt_func in self.child_nodes:
            child_node: Carrot
            child_node_grad = grad_wrt_func(
                grad
            )  # calculate partial grad, in the following bp, need to add partial grad
            child_node.backward(child_node_grad)
            pass
        pass

    def zero_grad(self) -> None:
        """
        zero grad
        """
        self.grad = 0
        for child_node, _ in self.child_nodes:
            child_node.zero_grad()
            pass
        pass

In [ ]:
x = Carrot(data=2.0, name="x")
x2 = x * x
g = x2 * x2
h = x2 * x2
y = g + h
y.backward() 
print("-------------------")
print(f"x={x.data}\tx2={x2.data}\tg={g.data}\th={h.data}\ty={y.data}")
print("-------------------")
print(f"dy/dy={y.grad}\tdy/dg={g.grad}\tdy/dh={h.grad}\tdy/dx2={x2.grad}\tdy/dx={x.grad}")
print("-------------------")
y.zero_grad()
g.backward()
print(f"dg/dx2={x2.grad}\tdg/dx={x.grad}")

In [19]:
x = Carrot(data=2.0, requires_grad=True, name="x")
y = Carrot(data=4.0, requires_grad=True, name="y")

z = x *  y
z.backward()
print(f"z.grad={z.grad}")
print(f"y.grad={y.grad}")
print(f"x.grad={x.grad}")
z.zero_grad()

a = Carrot(data=1.0, requires_grad=True, name="a")
coefficient1 = Carrot(data=2.0, requires_grad=False)
coefficient2 = Carrot(data=3.0, requires_grad=False)
z = a*coefficient1 + a * coefficient2
z.backward()
print(f"z.grad={z.grad}")
print(f"a.grad={a.grad}")
z.zero_grad()

z.grad=1.0
y.grad=2.0
x.grad=4.0
z.grad=1.0
a.grad=5.0


In [22]:
import numpy as np
a = np.array([1, 2, 3])
print(a.shape)

b = np.array([[1], [2], [3]])
print(b.shape)

(3,)
(3, 1)
